## GED conflict data clean up

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
##set a working directory
user = os.path.expanduser('~')
display(user)

os.makedirs(f'{user}/Desktop/thesis/', exist_ok=True)
path = os.chdir(f'{user}/Desktop/'+'thesis/')

'/Users/NatStrom'

In [3]:
data = pd.read_csv('GEDEvent_v22_1.csv')

/var/folders/3l/pgxm2f9j3bbgbjcj2pyxl__00000gp/T/ipykernel_34900/2663555733.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('GEDEvent_v22_1.csv')


In [4]:
df = data.copy()

In [5]:
print(df.dtypes)
print(df.columns)
print(df.describe(include='all'))

## sanity check to ensure the spread of the data is correct
print('sanity check 1, ensure appropriate years and countries are represented')
list_country=list(list(df['country_id'].drop_duplicates()))
list_year=list(list(df['year'].drop_duplicates()))
print('country list', list_country)
print('year list', list_year)
#inspect for NAs
print('following checks for completeness of all columns')
print(df['country_id'].isnull().values.any())

id                     int64
relid                 object
year                   int64
active_year            int64
code_status           object
type_of_violence       int64
conflict_dset_id       int64
conflict_new_id        int64
conflict_name         object
dyad_dset_id           int64
dyad_new_id            int64
dyad_name             object
side_a_dset_id         int64
side_a_new_id          int64
side_a                object
side_b_dset_id         int64
side_b_new_id          int64
side_b                object
number_of_sources      int64
source_article        object
source_office         object
source_date           object
source_headline       object
source_original       object
where_prec             int64
where_coordinates     object
where_description     object
adm_1                 object
adm_2                 object
latitude             float64
longitude            float64
geom_wkt              object
priogrid_gid           int64
country               object
country_id    

In [6]:
#filter only the wanted columns
df_filtered = df.filter(['id', 'relid', 'year', 'type_of_violence','conflict_new_id','country_id','best', 'high', 'low'])

In [7]:
display(df_filtered)

,id,relid,year,type_of_violence,conflict_new_id,country_id,best,high,low
0,244657,IRQ-2017-1-524-322,2017,1,259,700,6,6,6
1,412700,IRQ-2021-1-524-145,2021,1,259,700,183,184,171
2,413023,IRQ-2021-1-524-143,2021,1,259,700,2,3,0
3,412909,IRQ-2021-1-524-144,2021,1,259,700,10,10,9
4,132140,AFG-1989-1-411-2,1989,1,333,700,6,6,6
...,...,...,...,...,...,...,...,...,...
293629,15525,MZM-1989-3-1347-4,1989,3,562,552,9,9,9
293630,15524,MZM-1989-3-1347-16,1989,3,562,552,9,9,9
293631,15245,MZM-1990-3-1347-18,1990,3,562,552,7,7,7
293632,15516,MZM-1990-3-1347-28,1990,3,562,552,1,1,1


In [8]:
#merge with the brd dataset to prodce the lags
data_lag = pd.read_csv('brd.csv')

In [9]:
data_lag = data_lag.drop(columns={'Unnamed: 0', '...1', 'annualdata', 'source', 'bdversion', 'location', 'sidea', 'sidea2nd',
       'sideb', 'sideb2nd', 'incomp', 'terr', 'int', 'cumint','startdate', 'startprec', 'startdate2', 'startprec2', 'epend',
       'ependdate', 'ependprec', 'gwnoa', 'gwnoa2nd', 'gwnob', 'gwnob2nd',
       'gwnoloc'})
data_lag = data_lag.loc[data_lag['year'] >= 1970]

In [10]:
data_lag.columns
data_lag = data_lag.rename(columns={'id':'country_id','bdeadbes':'best'})

In [11]:
print(data_lag.dtypes)
print(df_filtered.dtypes)

country_id      int64
year            int64
bdeadlow        int64
bdeadhig      float64
best          float64
type            int64
region          int64
version        object
c_name         object
dtype: object
id                   int64
relid               object
year                 int64
type_of_violence     int64
conflict_new_id      int64
country_id           int64
best                 int64
high                 int64
low                  int64
dtype: object


In [13]:
df_filtered = pd.merge(df_filtered,data_lag,how='outer',on=['country_id','year','best'])
df_filtered

,id,relid,year,type_of_violence,conflict_new_id,country_id,best,high,low,bdeadlow,bdeadhig,type,region,version,c_name
0,244657.0,IRQ-2017-1-524-322,2017,1.0,259.0,700,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
1,237888.0,AFG-2017-1-735-260,2017,1.0,333.0,700,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2,236971.0,AFG-2017-1-327-15,2017,1.0,333.0,700,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
3,233589.0,AFG-2017-1-327-9,2017,1.0,333.0,700,6.0,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,237127.0,AFG-2017-1-735-2,2017,1.0,333.0,700,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295111,NaN,NaN,2007,NaN,NaN,257,56.0,NaN,NaN,30.0,100.0,3.0,1.0,2009-4,NaN
295112,NaN,NaN,2008,NaN,NaN,257,339.0,NaN,NaN,237.0,503.0,3.0,1.0,2009-4,NaN
295113,NaN,NaN,2008,NaN,NaN,258,61.0,NaN,NaN,35.0,61.0,3.0,3.0,2009-4,NaN
295114,NaN,NaN,2008,NaN,NaN,259,33.0,NaN,NaN,24.0,35.0,3.0,3.0,2009-4,NaN


In [15]:
#now produce the lags and filter out the unwanted years
#now I need to loop the lagging function, creating a new column and appending them all together to then have a dataframe i can concanate

df_filtered['lag_best_5'] = df_filtered.groupby('id')['best'].shift(5)
df_filtered['lag_best_10'] = df_filtered.groupby('id')['best'].shift(10)
df_filtered['lag_best_20'] = df_filtered.groupby('id')['best'].shift(20)
    

df_filtered = df_filtered.loc[df_filtered['year'] >= 1990]
df_filtered

,id,relid,year,type_of_violence,conflict_new_id,country_id,best,high,low,bdeadlow,bdeadhig,type,region,version,c_name,lag_best_5,lag_best_10,lag_best_20
0,244657.0,IRQ-2017-1-524-322,2017,1.0,259.0,700,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,237888.0,AFG-2017-1-735-260,2017,1.0,333.0,700,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,236971.0,AFG-2017-1-327-15,2017,1.0,333.0,700,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,233589.0,AFG-2017-1-327-9,2017,1.0,333.0,700,6.0,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,237127.0,AFG-2017-1-735-2,2017,1.0,333.0,700,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295111,NaN,NaN,2007,NaN,NaN,257,56.0,NaN,NaN,30.0,100.0,3.0,1.0,2009-4,NaN,NaN,NaN,NaN
295112,NaN,NaN,2008,NaN,NaN,257,339.0,NaN,NaN,237.0,503.0,3.0,1.0,2009-4,NaN,NaN,NaN,NaN
295113,NaN,NaN,2008,NaN,NaN,258,61.0,NaN,NaN,35.0,61.0,3.0,3.0,2009-4,NaN,NaN,NaN,NaN
295114,NaN,NaN,2008,NaN,NaN,259,33.0,NaN,NaN,24.0,35.0,3.0,3.0,2009-4,NaN,NaN,NaN,NaN


In [16]:
from ingester3.scratch import cache_manager

cache_manager()

# This will import the Pandas extensions as well as bootstrap the object layer
from ingester3.extensions import *

In [19]:
df_filtered = pd.DataFrame.c.from_gwcode(df_filtered, gw_col='country_id', month_col=None)

KeyError: 0

In [18]:
#create the c_id from the gw codes
#create c_ids
df_filtered = pd.DataFrame.cy.soft_validate_gwcode_year(df_filtered, gw_col='country_id', year_col = 'year', at_month = 12)
inspect=df_filtered.query("valid_id == False")
print('countries with invalid id below')
display(inspect)

df_filtered = pd.DataFrame.c.from_gwcode(df_filtered, gw_col='country_id', month_col=None)

countries with invalid id below


,id,relid,year,type_of_violence,conflict_new_id,country_id,best,high,low,bdeadlow,bdeadhig,type,region,version,c_name,lag_best_5,lag_best_10,lag_best_20,valid_id
293644,NaN,NaN,1990,NaN,NaN,6,50.0,NaN,NaN,25.0,999.0,3.0,2.0,2009-4,NaN,NaN,NaN,NaN,False
293645,NaN,NaN,1993,NaN,NaN,6,50.0,NaN,NaN,50.0,200.0,3.0,2.0,2009-4,NaN,NaN,NaN,NaN,False
293646,NaN,NaN,1996,NaN,NaN,6,27.0,NaN,NaN,25.0,999.0,3.0,2.0,2009-4,NaN,NaN,NaN,NaN,False
293667,NaN,NaN,1990,NaN,NaN,10,405.0,NaN,NaN,25.0,999.0,3.0,3.0,2009-4,NaN,NaN,NaN,NaN,False
293668,NaN,NaN,1991,NaN,NaN,10,-999.0,NaN,NaN,1200.0,1500.0,3.0,3.0,2009-4,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295109,NaN,NaN,2008,NaN,NaN,255,47.0,NaN,NaN,25.0,999.0,3.0,4.0,2009-4,Germany (Prussia),NaN,NaN,NaN,False
295111,NaN,NaN,2007,NaN,NaN,257,56.0,NaN,NaN,30.0,100.0,3.0,1.0,2009-4,NaN,NaN,NaN,NaN,False
295112,NaN,NaN,2008,NaN,NaN,257,339.0,NaN,NaN,237.0,503.0,3.0,1.0,2009-4,NaN,NaN,NaN,NaN,False
295113,NaN,NaN,2008,NaN,NaN,258,61.0,NaN,NaN,35.0,61.0,3.0,3.0,2009-4,NaN,NaN,NaN,NaN,False


KeyError: 0

In [ ]:
df_filtered['Africa'] = df_filtered.c.in_africa

In [ ]:
df_filtered = df_filtered.query('Africa == True')

In [ ]:
#get the iso numbers
df_filtered['iso'] = df_filtered.c.isoab

In [ ]:
inspect = df_filtered.loc[df_filtered.iso.isna()]
display(inspect)

In [ ]:
df_conflict = df_filtered.copy()

In [ ]:
df_conflict = df_conflict.filter(['iso','year','type_of_violence', 'best', 'high', 'low'])
df_conflict

In [17]:
#replace row values
df_conflict['type_of_violence'] = df_conflict['type_of_violence'].map({1:'sb',
                                            2:'ns',
                                            3:'os'})

In [18]:
#create dummies
def categorize(row):  
    if row['type_of_violence'] == 'sb':
        return 1
    return 0

df_conflict['dummy_sb'] = df_conflict.apply(lambda row: categorize(row), axis=1)

# repeat for non-state violence
def categorize(row):  
    if row['type_of_violence'] == 'ns':
        return 1
    return 0

df_conflict['dummy_ns'] = df_conflict.apply(lambda row: categorize(row), axis=1)

#repeat dummy creation for one-sided violence
def categorize(row):  
    if row['type_of_violence'] == 'os':
        return 1
    return 0

df_conflict['dummy_os'] = df_conflict.apply(lambda row: categorize(row), axis=1)



In [19]:
#add dummies for the type of violence
df_conflict

,iso,year,type_of_violence,best,high,low,dummy_sb,dummy_ns,dummy_os
41447,DZA,1990,sb,1,1,1,1,0,0
41448,DZA,1991,sb,3,3,3,1,0,0
41449,DZA,1991,sb,4,4,4,1,0,0
41450,DZA,1991,sb,10,10,10,1,0,0
41451,DZA,1991,sb,17,17,17,1,0,0
...,...,...,...,...,...,...,...,...,...
293629,ZWE,1989,os,9,9,9,0,0,1
293630,ZWE,1989,os,9,9,9,0,0,1
293631,ZWE,1990,os,7,7,7,0,0,1
293632,ZWE,1990,os,1,1,1,0,0,1


In [21]:
#create dummies for intensity
def categorize(row):  
    if row['best'] <= 100:
        return 'low'
    if row['best'] > 100 or row['best'] < 1000:
        return 'medium'
    return 'high'

df_conflict['intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)
df_conflict

,iso,year,type_of_violence,best,high,low,dummy_sb,dummy_ns,dummy_os,intensity
41447,DZA,1990,sb,1,1,1,1,0,0,low
41448,DZA,1991,sb,3,3,3,1,0,0,low
41449,DZA,1991,sb,4,4,4,1,0,0,low
41450,DZA,1991,sb,10,10,10,1,0,0,low
41451,DZA,1991,sb,17,17,17,1,0,0,low
...,...,...,...,...,...,...,...,...,...,...
293629,ZWE,1989,os,9,9,9,0,0,1,low
293630,ZWE,1989,os,9,9,9,0,0,1,low
293631,ZWE,1990,os,7,7,7,0,0,1,low
293632,ZWE,1990,os,1,1,1,0,0,1,low


In [22]:
# create dummy for low intensity conflict
def categorize(row):  
    if row['intensity'] == 'low':
        return 1
    return 0

df_conflict['dummy_low_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

#repeat dummy creation for medium intensity
def categorize(row):  
    if row['intensity'] == 'middle':
        return 1
    return 0

df_conflict['dummy_medium_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

def categorize(row):  
    if row['intensity'] == 'high':
        return 1
    return 0

df_conflict['dummy_high_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

In [23]:
df_conflict = df_conflict.loc[df_conflict['year'] <= 2019]

In [24]:
df_conflict = df_conflict.loc[df_conflict['year'] >= 1990]

In [25]:
df_conflict

,iso,year,type_of_violence,best,high,low,dummy_sb,dummy_ns,dummy_os,intensity,dummy_low_intensity,dummy_medium_intensity,dummy_high_intensity
41447,DZA,1990,sb,1,1,1,1,0,0,low,1,0,0
41448,DZA,1991,sb,3,3,3,1,0,0,low,1,0,0
41449,DZA,1991,sb,4,4,4,1,0,0,low,1,0,0
41450,DZA,1991,sb,10,10,10,1,0,0,low,1,0,0
41451,DZA,1991,sb,17,17,17,1,0,0,low,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293626,ZWE,2019,os,1,1,1,0,0,1,low,1,0,0
293627,ZWE,2019,os,2,2,2,0,0,1,low,1,0,0
293631,ZWE,1990,os,7,7,7,0,0,1,low,1,0,0
293632,ZWE,1990,os,1,1,1,0,0,1,low,1,0,0


In [28]:
#create the neighbouring country at war index, best >= 25
#first try it for one country
dza_neighbors = Country.from_iso('DZA', month_id=455).neighbors()

print('dza neighbour ID: ', dza_neighbors)
print()
print('dza neighbour names:', [i.name for i in dza_neighbors])

Russian Neighbors ID:  [Country(50), Country(73), Country(78), Country(213), Country(243), Country(244)]

Russian Neighbors Names: ['Mali', 'Tunisia', 'Niger', 'Libya', 'Morocco', 'Mauritania']


In [40]:
list_c = list(df_conflict.iso.unique())

In [47]:
neighbors = list()
for i in list_c:
    neighbors = Country.from_iso(i,month_id =455).neighbors()
    neighbors_all = neighbors.append()

TypeError: list.append() takes exactly one argument (0 given)

In [42]:
neighbors

[Country(154),
 Country(157),
 Country(162),
 Country(163),
 Country(170),
 Country(192)]

In [ ]:
#produce 5, 10 and 20 year lag

In [26]:
df_conflict.to_csv('df_conflict.csv')